In [1]:
#문서 path 추출
import pandas as pd
import os
import re
import json
import numpy as np

filenames = []
directory = '/Users/minjikim/Desktop/2021년 하반기 민지 휴학/하울스/제주도 방언 /training/[라벨]제주도_학습용데이터_1 (1)'
for filename in os.listdir(directory):
    #filename이 .json으로 끝나면
    if filename.endswith(".json"):
        #f 생성
        f = directory + '/' + filename
        #비어있는 filenames에 append
        filenames.append(f)

print(filenames[:5])
print("파일 개수:", len(filenames))

['/Users/minjikim/Desktop/2021년 하반기 민지 휴학/하울스/제주도 방언 /training/[라벨]제주도_학습용데이터_1 (1)/DZHF20002969.json', '/Users/minjikim/Desktop/2021년 하반기 민지 휴학/하울스/제주도 방언 /training/[라벨]제주도_학습용데이터_1 (1)/DZHF20000954.json', '/Users/minjikim/Desktop/2021년 하반기 민지 휴학/하울스/제주도 방언 /training/[라벨]제주도_학습용데이터_1 (1)/DZJD21001992.json', '/Users/minjikim/Desktop/2021년 하반기 민지 휴학/하울스/제주도 방언 /training/[라벨]제주도_학습용데이터_1 (1)/DZJD20000781.json', '/Users/minjikim/Desktop/2021년 하반기 민지 휴학/하울스/제주도 방언 /training/[라벨]제주도_학습용데이터_1 (1)/DZHF20002590.json']
파일 개수: 5042


In [2]:
#방언, 표준어 추출
standard= []
dialect = []
for file in filenames:
    with open(file, "r", encoding = 'utf8') as f:
        contents = f.read() #string type 파일 내용 전체를 문자열로
        json_data = json.loads(contents)
        doc = json_data["utterance"]
        for d in doc:
            standard.append(d['standard_form'])
            dialect.append(d['dialect_form'])

In [3]:
print(standard[:5])
print(dialect[:5])
print(len(standard))
print(len(dialect))

['왜냐하면 이만한 그', '-이만- 이만한 크기.', '응.', '제사 때 쓰는 거.', '아니 게메 저 차판 자체가 쓸 일이 많지가 않을 거 같아 #이 거기서 #이 왜냐하면']
['왜냐하면 이만한 그', '-이만- 이만한 크기.', '응.', '제사 때 쓰는 거.', '아니 게메 저 차판 자체가 쓸 일이 하지가 않을 거 닮아 이 거기서 이 왜냐하면']
2774257
2774257


In [4]:
#dataframe
train = pd.DataFrame({'standard': standard, 'dialect': dialect})

In [5]:
train[:5]

,standard,dialect
0,왜냐하면 이만한 그,왜냐하면 이만한 그
1,-이만- 이만한 크기.,-이만- 이만한 크기.
2,응.,응.
3,제사 때 쓰는 거.,제사 때 쓰는 거.
4,아니 게메 저 차판 자체가 쓸 일이 많지가 않을 거 같아 #이 거기서 #이 왜냐하면,아니 게메 저 차판 자체가 쓸 일이 하지가 않을 거 닮아 이 거기서 이 왜냐하면


In [6]:
print(len(standard))

2774257


In [7]:
'''
전처리하면서 살펴봐야 하는 목록
1) {laughing}, {cleaning} 등의 비언어적 표현
2) (이)/(#이) 와 같은 표현은 아예 삭제해도 문제 없는 듯함 
3) 2774255에서 보는 바와 같이 (방언 전사 형태)/(표준어 대응쌍 형태)로 구성 되어 있음 
4) 정보 비식별화를 위해 &name&, &address& 등으로 되어 있음
5) 아래아는 아*  형태로 되어 있음
6) (()), ((x)), ((xxxx)) 등의 형태 
'''

'\n전처리하면서 살펴봐야 하는 목록\n1) {laughing}, {cleaning} 등의 비언어적 표현\n2) (이)/(#이) 와 같은 표현은 아예 삭제해도 문제 없는 듯함 \n3) 2774255에서 보는 바와 같이 (방언 전사 형태)/(표준어 대응쌍 형태)로 구성 되어 있음 \n4) 정보 비식별화를 위해 &name&, &address& 등으로 되어 있음\n5) 아래아는 아*  형태로 되어 있음\n6) (()), ((x)), ((xxxx)) 등의 형태 \n'

In [8]:
# Standard_form
# 1번 2번 삭제
pre_s = []
for sentence in standard:
    sentence_new = re.sub(r'\{.+\}', '', sentence) #{laughing}, {clearing} 삭제 
    sentence_new = re.sub('\(이\)/\(#이\)', '', sentence_new) # (이)/(#이) 삭제
    pre_s.append(sentence_new)

In [9]:
# standard는 두 번째 그룹 선택 
pre_standard = []
for snts in pre_s:  #(방언)/(표준어)형태의 문장이 있을 경우 (표준어)를 선택하도록 함
    if re.search('(\([\s]?[가-힣]+[\.\?\s\*]?\))/(\([\s#]?[가-힣]+[\.\?\s\*]?\))', snts):
        m = re.search('(\([\s]?[가-힣]+[\.\?\s\*]?\))/(\([\s#]?[가-힣]+[\.\?\s\*]?\))', snts)
        #snts에서 (방언)/(표준어)를 찾고 이를 group(1), group(2)로 보는 것 
        replace_str = re.sub('[()]','',m.group(2)) #(표준어)를 표준어로 바꾸는 작업
        sentence_new = re.sub('(\([\s]?[가-힣]+[\.\?\s\*]?\))/(\([\s#]?[가-힣]+[\.\?\s\*]?\))', replace_str, snts)
        
        pre_standard.append(sentence_new)
    
    else: #(방언)/(표준어) 형태의 expression이 없을 때는 원래의 문장을 바로 pre_standard에 append
        pre_standard.append(snts) 

In [10]:
# Dialect_form
# 1번 2번 삭제
pre_d = []
for sentence in dialect:
    sentence_new = re.sub(r'\{.+\}', '', sentence) #{laughing}, {clearing} 삭제 
    sentence_new = re.sub('\(이\)/\(#이\)', '', sentence_new) # (이)/(#이) 삭제
    pre_d.append(sentence_new)

In [11]:
# dialect는 첫 번째 그룹 선택 
pre_dialect = []
for snts in pre_d:  #(방언)/(표준어)형태의 문장이 있을 경우 (방언)을 선택하도록 함
    if re.search('(\([\s]?[가-힣]+[\.\?\s\*]?\))/(\([\s#]?[가-힣]+[\.\?\s\*]?\))', snts):
        m = re.search('(\([\s]?[가-힣]+[\.\?\s\*]?\))/(\([\s#]?[가-힣]+[\.\?\s\*]?\))', snts)
        #snts에서 (방언)/(표준어)를 찾고 이를 group(1), group(2)로 보는 것 
        replace_str = re.sub('[()]','',m.group(1)) #(표준어)를 표준어로 바꾸는 작업
        sentence_new = re.sub('(\([\s]?[가-힣]+[\.\?\s\*]?\))/(\([\s#]?[가-힣]+[\.\?\s\*]?\))', replace_str, snts)
        
        pre_dialect.append(sentence_new)
    
    else: #(방언)/(표준어) 형태의 expression이 없을 때는 원래의 문장을 바로 pre_standard에 append
        pre_dialect.append(snts) 

In [12]:
print(len(pre_s))
print(len(pre_d))

2774257
2774257


In [13]:
print(len(pre_standard))
print(len(pre_dialect))

2774257
2774257


In [15]:
preprocessed_train = pd.DataFrame({'standard': pre_standard, 'dialect': pre_dialect})

In [16]:
preprocessed_train

,standard,dialect
0,왜냐하면 이만한 그,왜냐하면 이만한 그
1,-이만- 이만한 크기.,-이만- 이만한 크기.
2,응.,응.
3,제사 때 쓰는 거.,제사 때 쓰는 거.
4,아니 게메 저 차판 자체가 쓸 일이 많지가 않을 거 같아 #이 거기서 #이 왜냐하면,아니 게메 저 차판 자체가 쓸 일이 하지가 않을 거 닮아 이 거기서 이 왜냐하면
...,...,...
2774252,지금 우리가 시간이 열 한 시 사십 분이 넘었나? 사십 분 되니까,지금 우리가 시간이 열 한 시 사십 분이 넘었나? 사십 분 되난
2774253,으응 사십분 - 내가 삼십 분에 끝나서 사십 분에 통화 하기로 해 가지고,으응 사십분 - 내가 삼십 분에 끝낭 사십 분에 통화 하기로 해 가지고
2774254,전화 오는 거 같은데,전화 오는 거 닮은디
2774255,응~,응~


In [17]:
preprocessed_train.to_csv("jeju_train_prepd.csv", index = False, encoding = "utf-8-sig")

validation 전처리

In [18]:
#문서 path 추출
import pandas as pd
import os
import re
import json
import numpy as np

filenames = []
directory = '/Users/minjikim/Desktop/2021년 하반기 민지 휴학/하울스/제주도 방언 /test/[라벨]제주도_학습용데이터_3 (1)'
for filename in os.listdir(directory):
    #filename이 .json으로 끝나면
    if filename.endswith(".json"):
        #f 생성
        f = directory + '/' + filename
        #비어있는 filenames에 append
        filenames.append(f)

print(filenames[:5])
print("파일 개수:", len(filenames))

['/Users/minjikim/Desktop/2021년 하반기 민지 휴학/하울스/제주도 방언 /test/[라벨]제주도_학습용데이터_3 (1)/DZJD21002347.json', '/Users/minjikim/Desktop/2021년 하반기 민지 휴학/하울스/제주도 방언 /test/[라벨]제주도_학습용데이터_3 (1)/DZHF20001216.json', '/Users/minjikim/Desktop/2021년 하반기 민지 휴학/하울스/제주도 방언 /test/[라벨]제주도_학습용데이터_3 (1)/DZJD20000294.json', '/Users/minjikim/Desktop/2021년 하반기 민지 휴학/하울스/제주도 방언 /test/[라벨]제주도_학습용데이터_3 (1)/DZJD20000017.json', '/Users/minjikim/Desktop/2021년 하반기 민지 휴학/하울스/제주도 방언 /test/[라벨]제주도_학습용데이터_3 (1)/DZES20001427.json']
파일 개수: 596


In [19]:
#방언, 표준어 추출
standard= []
dialect = []
for file in filenames:
    with open(file, "r", encoding = 'utf8') as f:
        contents = f.read() #string type 파일 내용 전체를 문자열로
        json_data = json.loads(contents)
        doc = json_data["utterance"]
        for d in doc:
            standard.append(d['standard_form'])
            dialect.append(d['dialect_form'])

In [20]:
#dataframe
train = pd.DataFrame({'standard': standard, 'dialect': dialect})

In [21]:
# Standard_form
# 1번 2번 삭제
pre_s = []
for sentence in standard:
    sentence_new = re.sub(r'\{.+\}', '', sentence) #{laughing}, {clearing} 삭제 
    sentence_new = re.sub('\(이\)/\(#이\)', '', sentence_new) # (이)/(#이) 삭제
    pre_s.append(sentence_new)
    


In [22]:
# standard는 두 번째 그룹 선택 
pre_standard = []
for snts in pre_s:  #(방언)/(표준어)형태의 문장이 있을 경우 (표준어)를 선택하도록 함
    if re.search('(\([\s]?[가-힣]+[\.\?\s\*]?\))/(\([\s#]?[가-힣]+[\.\?\s\*]?\))', snts):
        m = re.search('(\([\s]?[가-힣]+[\.\?\s\*]?\))/(\([\s#]?[가-힣]+[\.\?\s\*]?\))', snts)
        #snts에서 (방언)/(표준어)를 찾고 이를 group(1), group(2)로 보는 것 
        replace_str = re.sub('[()]','',m.group(2)) #(표준어)를 표준어로 바꾸는 작업
        sentence_new = re.sub('(\([\s]?[가-힣]+[\.\?\s\*]?\))/(\([\s#]?[가-힣]+[\.\?\s\*]?\))', replace_str, snts)
        
        pre_standard.append(sentence_new)
    
    else: #(방언)/(표준어) 형태의 expression이 없을 때는 원래의 문장을 바로 pre_standard에 append
        pre_standard.append(snts) 

In [23]:
# Dialect_form
# 1번 2번 삭제
pre_d = []
for sentence in dialect:
    sentence_new = re.sub(r'\{.+\}', '', sentence) #{laughing}, {clearing} 삭제 
    sentence_new = re.sub('\(이\)/\(#이\)', '', sentence_new) # (이)/(#이) 삭제
    pre_d.append(sentence_new)

In [24]:
# dialect는 첫 번째 그룹 선택 
pre_dialect = []
for snts in pre_d:  #(방언)/(표준어)형태의 문장이 있을 경우 (방언)을 선택하도록 함
    if re.search('(\([\s]?[가-힣]+[\.\?\s\*]?\))/(\([\s#]?[가-힣]+[\.\?\s\*]?\))', snts):
        m = re.search('(\([\s]?[가-힣]+[\.\?\s\*]?\))/(\([\s#]?[가-힣]+[\.\?\s\*]?\))', snts)
        #snts에서 (방언)/(표준어)를 찾고 이를 group(1), group(2)로 보는 것 
        replace_str = re.sub('[()]','',m.group(1)) #(표준어)를 표준어로 바꾸는 작업
        sentence_new = re.sub('(\([\s]?[가-힣]+[\.\?\s\*]?\))/(\([\s#]?[가-힣]+[\.\?\s\*]?\))', replace_str, snts)
        
        pre_dialect.append(sentence_new)
    
    else: #(방언)/(표준어) 형태의 expression이 없을 때는 원래의 문장을 바로 pre_standard에 append
        pre_dialect.append(snts) 

In [25]:
preprocessed_train = pd.DataFrame({'standard': pre_standard, 'dialect': pre_dialect})

In [26]:
preprocessed_train

,standard,dialect
0,아 예 저는 &name1&입니다,아 예 저는 &name1&입니다
1,저는 &name2&입니다,저는 &name2&입니다
2,네,네
3,이제 시작인가 봐,이제 시작인가 봐
4,어 왜 녹 녹음뜨지,어 왜 녹 녹음뜨멘
...,...,...
333797,어쨌든 다음주 중에는 빨리 식장은 결정해야겠다,어쨌든 다음주 중에는 빨리 식장은 결정해야겠다
333798,((xxxx))는 그냥 아예 빼버리고,((xxxx))는 그냥 아예 빼버리고
333799,왜? 거기(()),왜? 거기(())
333800,대관료가 왜 난 백만원으로 알고 있지?,대관료가 왜 난 백만원으로 알고 있지?


In [27]:
preprocessed_train.to_csv("jeju_val_prepd.csv", index = False, encoding = "utf-8-sig")